# 2. Download Lasair light curves

Uses the **Lasair API** to fetch light-curve data for each object in `ztf_cleansed.csv`. You choose a **run folder** name; the notebook creates `runs/<run_name>/` and writes one CSV per object: `runs/<run>/<ZTFID>_lightcurve.csv`.

Each file contains MJD, filter (g/r), and flux/errors. When forced flux is missing, **`utils.processLasairData()`** fills `forced_ujy` and `forced_ujy_error` from unforced magnitude (AB zeropoint 23.9) and then cleans and overwrites the CSV.

In [ ]:
# load ztf data
from pathlib import Path
from datetime import datetime
import pandas as pd
import lasair
import numpy as np
import shutil
from utils import processLasairData

project_root = Path.cwd().parent
data_folder = project_root / "data"
data_folder.mkdir(parents=True, exist_ok=True)

# create specific folder for this run
folder_name = input("Enter name for run folder (leave blank for timestamp): ").strip()
if not folder_name:
    start_time = datetime.now()
    folder_name = start_time.strftime("%Y-%m-%d_%H-%M-%S")
run_folder = project_root / "runs" / folder_name
run_folder.mkdir(parents=True, exist_ok=True)

# reade the cleansed data
ztf_data = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_data)
print(len(ztf_df))
print(ztf_df.head())

Enter name for run folder (leave blank for timestamp): run3
5004
          ZTFID      IAUID           RA          Dec     peakt peakfilt  \
0  ZTF17aabtvsy  SN2022yei  10:35:32.09  +37:38:59.0  59870.49        r   
1  ZTF17aabvong  SN2024xxq  02:05:07.68  +11:14:55.1  60606.25        g   
2  ZTF17aacldgo  SN2022zxv  03:09:24.36  -04:53:39.2  59897.25        g   
3  ZTF17aadlxmv  SN2020adv  08:29:47.59  +33:54:22.8  58879.19        g   
4  ZTF18aaaonon  SN2022jjs  10:19:05.51  +14:24:16.6  59703.27        g   

   peakmag  peakabs duration    rise      fade   type  redshift          b  \
0  18.0303   -19.41  >34.229   >6.01    28.219  SN Ia   0.06922  59.641962   
1  16.8039   -19.57   23.222   8.464    14.758  SN Ia   0.03400 -47.664064   
2  18.7979   -18.91    >1077   >3.85  >1073.15  SN Ia   0.07200 -50.332472   
3  17.9475   -19.34   25.146  10.951    14.195  SN Ia   0.06200  34.174702   
4  18.5663   -19.08  >18.631   3.691    >14.94  SN Ia   0.07141  52.363911   

     A_V  host_

In [ ]:
from typing import Any

# lasair api token
lasair_token = 'c4ce6509f05363cfb57aaedffb056ef31573e647'
client = lasair.lasair_client(lasair_token)
print("Lasair client initialized.")

num_total = len(ztf_df)
num_processed = 0
num_success = 0

for idx, row in enumerate[Any](ztf_df.itertuples(index=False)):
    num_processed += 1
    print(f"\nProcessing {num_processed:3d}/{num_total}: {row.ZTFID}...", end='')
    ztf_id = row.ZTFID
    iauid = row.IAUID
    if str(iauid).startswith('SN'):
        iauid = str(iauid)[2:]

    data_path = data_folder / f"{ztf_id}_lightcurve.csv"
    run_path = run_folder / f"{ztf_id}_lightcurve.csv"
    print(f" Checking data folder...", end='')
    # we don't need to keep downlaoding the same data. skip if already stored.
    if data_path.exists():
        print(f" Already exists.", end='')
        df_obj = pd.read_csv(data_path)
    else:
        print(f" Downloading...", end='')
        try:
            result = client.object(ztf_id)
        except Exception as e:
            print(f"\nLasair API error for {ztf_id}: {e}")
            continue

        if not result or 'candidates' not in result:
            print(f"\nNo candidates found for {ztf_id}. The object may not exist or API error.")
            continue
        if "error" in result:
            print(f"\nAPI returned error for {ztf_id}: {result.get('error')}")
            continue

        candidates = result['candidates']
        if not candidates:
            print(f"\nNo light curve data for {ztf_id}")
            continue

        output_rows = []
        for cand in candidates:
            # fid: filter ID (1=g, 2=r)
            fid = cand.get("fid")
            # keep i band for completeness             
            fid_to_letter = {1: "g", 2: "r", 3: "i"}
            filter_letter = fid_to_letter.get(fid, fid) if fid is not None else None
            
            # store light curve data as new csv
            out_row = {
                'ztf_id': ztf_id,
                'iauid': iauid,
                'MJD': cand.get('mjd', None),
                'filter': filter_letter,
                'unforced_mag': cand.get('magpsf', None),
                'unforced_mag_error': cand.get('sigmapsf', None),
                'unforced_mag_status': 'positive' if cand.get('isdiffpos', 't') == 't' else 'negative',
                'forced_ujy': cand.get('forcediffimflux', None),
                'forced_ujy_error': cand.get('forcediffimfluxunc', None),
            }
            output_rows.append(out_row)

        df_obj = pd.DataFrame(output_rows)
        df_obj.to_csv(data_path, index=False)
        print(f" Saved {len(output_rows)} rows.", end='')
    # move a copy of the lightcurve into the run folder
    print(f" Copying to run folder...", end='')
    shutil.copy(data_path, run_path)
    print(f" Copied.", end='')
    
    print(f" Processing...", end='')
    # we process the csv, ensuring they have forced_ujy and forced_ujy_error
    try:
        processLasairData(df_obj, run_path)
        num_success += 1
        print(f" Done ({num_success}/{num_processed})")
    except Exception as e:
        print(f" Error: {e} ({num_success}/{num_processed})")

print(f"\nAll done! {num_success} of {num_total} objects processed successfully.")

Lasair client initialized.

Processing   1/5004: ZTF17aabtvsy... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF17aabtvsy_lightcurve.csv (23 rows)
 Done (1/1)

Processing   2/5004: ZTF17aabvong... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF17aabvong_lightcurve.csv (31 rows)
 Done (2/2)

Processing   3/5004: ZTF17aacldgo... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF17aacldgo_lightcurve.csv (17 rows)
 Done (3/3)

Processing   4/5004: ZTF17aadlxmv... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/

Processing  34/5004: ZTF18aamzgzi... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aamzgzi_lightcurve.csv (56 rows)
 Done (34/34)

Processing  35/5004: ZTF18aangpkx... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aangpkx_lightcurve.csv (30 rows)
 Done (35/35)

Processing  36/5004: ZTF18aanhpii... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aanhpii_lightcurve.csv (13 rows)
 Done (36/36)

Processing  37/5004: ZTF18aannern... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aannern_ligh

Processing  75/5004: ZTF18aauizcr... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aauizcr_lightcurve.csv (392 rows)
 Done (75/75)

Processing  76/5004: ZTF18aaunfqq... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aaunfqq_lightcurve.csv (18 rows)
 Done (76/76)

Processing  77/5004: ZTF18aauotcv... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aauotcv_lightcurve.csv (26 rows)
 Done (77/77)

Processing  78/5004: ZTF18aavdwmh... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aavdwmh_lig

Processing 119/5004: ZTF18aayyqia... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aayyqia_lightcurve.csv (15 rows)
 Done (119/119)

Processing 120/5004: ZTF18aazabmh... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aazabmh_lightcurve.csv (16 rows)
 Done (120/120)

Processing 121/5004: ZTF18aazblzy... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aazblzy_lightcurve.csv (36 rows)
 Done (121/121)

Processing 122/5004: ZTF18aazhwnh... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18aazhwn

Processing 166/5004: ZTF18abffdjl... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18abffdjl_lightcurve.csv (27 rows)
 Done (166/166)

Processing 167/5004: ZTF18abffdpa... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18abffdpa_lightcurve.csv (59 rows)
 Done (167/167)

Processing 168/5004: ZTF18abfgygp... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18abfgygp_lightcurve.csv (45 rows)
 Done (168/168)

Processing 169/5004: ZTF18abfhaji... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF18abfhaj

...

Processing 4821/5004: ZTF25abpkect... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25abpkect_lightcurve.csv (50 rows)
 Done (4820/4821)

Processing 4822/5004: ZTF25abpopdk... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25abpopdk_lightcurve.csv (28 rows)
 Done (4821/4822)

Processing 4823/5004: ZTF25abptfih... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25abptfih_lightcurve.csv (39 rows)
 Done (4822/4823)

Processing 4824/5004: ZTF25abpuuql... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/Z

Processing 4864/5004: ZTF25abvcnys... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25abvcnys_lightcurve.csv (17 rows)
 Done (4863/4864)

Processing 4865/5004: ZTF25abvepyj... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25abvepyj_lightcurve.csv (40 rows)
 Done (4864/4865)

Processing 4866/5004: ZTF25abvpmki... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25abvpmki_lightcurve.csv (88 rows)
 Done (4865/4866)

Processing 4867/5004: ZTF25abvqhcq... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/Z

Processing 4907/5004: ZTF25acavlfh... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25acavlfh_lightcurve.csv (15 rows)
 Done (4906/4907)

Processing 4908/5004: ZTF25acaypgx... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25acaypgx_lightcurve.csv (21 rows)
 Done (4907/4908)

Processing 4909/5004: ZTF25acayrnp... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25acayrnp_lightcurve.csv (37 rows)
 Done (4908/4909)

Processing 4910/5004: ZTF25acbduge... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/Z

Processing 4950/5004: ZTF25acfwyhj... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25acfwyhj_lightcurve.csv (45 rows)
 Done (4949/4950)

Processing 4951/5004: ZTF25acfxafr... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25acfxafr_lightcurve.csv (39 rows)
 Done (4950/4951)

Processing 4952/5004: ZTF25acfxgpq... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25acfxgpq_lightcurve.csv (25 rows)
 Done (4951/4952)

Processing 4953/5004: ZTF25acfxytw... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/Z

Processing 4995/5004: ZTF25ackdapv... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25ackdapv_lightcurve.csv (38 rows)
 Done (4994/4995)

Processing 4996/5004: ZTF25ackmpgh... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25ackmpgh_lightcurve.csv (15 rows)
 Done (4995/4996)

Processing 4997/5004: ZTF25ackrgcs... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/ZTF25ackrgcs_lightcurve.csv (20 rows)
 Done (4996/4997)

Processing 4998/5004: ZTF25ackwqbt... Checking data folder... Already exists. Copying to run folder... Copied. Processing...Successfully cleaned and wrote light curve data to /Users/david/Code/msc/runs/run3/Z